In [1]:
%load_ext autoreload
%autoreload 2

# 2. Logistic Regression
## 2.5. Triển khai Logistic Regression
### 2.5.1. Ví dụ:
* Xây dựng logistic model để dự đoán `delay` (giá trị classification là chuyến bay có bị delay hay ko) từ `mon` (khoảng không gian cất cánh an toàn), `dom` (trong một tháng sẽ có bao nhiêu chuyến bay đến nơi dc chỉ định), `dow` (một tuần sẽ có bao nhiêu chuyến bay đến nơi dc chỉ định), `carrier` (hãng vận tải), `org` (nơi xuất phát), `km` (= `mile` * 1.60934) (độ dài đường bay), `depart` (thời gian khởi hành), `duration` (thời gian bay).


In [2]:
from modules.my_pyspark import *

In [3]:
spark = MyPySpark(session=True)

### 2.5.2. Đọc dữ liệu

In [4]:
data_path = r"./data/flights.csv"

In [5]:
data = spark.readFile(data_path, 'csv')

In [6]:
data.show(3, False)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|11 |20 |6  |US     |19    |JFK|2153|9.48  |351     |NA   |
|0  |22 |2  |UA     |1107  |ORD|316 |16.33 |82      |30   |
|2  |20 |4  |UA     |226   |SFO|337 |6.17  |82      |-8   |
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 3 rows



In [7]:
data.describe().toPandas()

,summary,mon,dom,dow,carrier,flight,org,mile,depart,duration,delay
0,count,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000
1,mean,5.2351,15.66196,2.95236,None,2054.31344,None,882.40112,14.130952600000064,151.76582,28.663795670111863
2,stddev,3.437758623534696,8.772488135606777,1.966033503314405,None,2182.4715300582875,None,701.232785607705,4.694052286573998,87.04507290261697,54.24034253403119
3,min,0,1,0,AA,1,JFK,67,0.25,30,-1
4,max,11,31,6,WN,6941,TUS,4243,23.98,560,NA


### 2.5.3. Chuẩn hóa dữ liệu
* Tạo cột `km` từ `mile`

In [8]:
from pyspark.sql.functions import round

In [9]:
data = data.withColumn('km', round(data['mile'] * 1.60934, 0))

In [10]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|
+---+---+---+-------+------+---+----+------+--------+-----+------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|
+---+---+---+-------+------+---+----+------+--------+-----+------+
only showing top 3 rows



* Tạo cột `label` tham chiếu từ cột `delay` với giá trị $1$ là có delay và $0$ là ko delay, quy ước `delay` $\geq 15 \text{ phút} \Rightarrow$ có delay.

In [11]:
data = data.withColumn('label', (data['delay'] >= 15).cast('integer'))

In [12]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 3 rows



* Chuẩn cột `carrier`, `org` thành dữ liệu số

In [13]:
import modules.my_spark_preprocessing as SP

In [14]:
data = SP.oneHotEncoder(data, ['carrier', 'org'])

In [15]:
data.toPandas().iloc[:3, :]

,mon,dom,dow,carrier,flight,org,mile,depart,duration,delay,km,label,idx_carrier,idx_org,oh_carrier,oh_org
0,11,20,6,US,19,JFK,2153,9.48,351,NA,3465.0,NaN,6.0,2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
1,0,22,2,UA,1107,ORD,316,16.33,82,30,509.0,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,2,20,4,UA,226,SFO,337,6.17,82,-8,542.0,0.0,0.0,1.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"


### 2.5.4. Chuyển đổi dữ liệu

In [47]:
from modules.my_spark_logistic_regression import *

In [17]:
predictor_features = ['mon', 'dom', 'dow', 'idx_carrier', 'idx_org', 'km', 'depart', 'duration']
target_feature = 'label'

In [18]:
lor_model = MySparkLogisticRegression(data, predictor_features, target_feature)

In [19]:
lor_model.prepareData()

In [20]:
lor_model.df_data.show(5, False)

+-----------------------------------------+-----+
|features                                 |label|
+-----------------------------------------+-----+
|[11.0,20.0,6.0,6.0,2.0,3465.0,9.48,351.0]|null |
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |1    |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |0    |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|0    |
|[4.0,2.0,5.0,1.0,0.0,415.0,8.92,65.0]    |null |
+-----------------------------------------+-----+
only showing top 5 rows



* Xóa các sample null

In [21]:
lor_model.df_data = SP.dropNa(lor_model.df_data)

In [22]:
lor_model.df_data.count()

47022

In [23]:
data.count()

50000

### 2.5.5. Chia dữ liệu thành train/test data

In [24]:
lor_model.trainTestSplit(.8)

In [25]:
lor_model.train_data.show(3, False)

+------------------------------------+-----+
|features                            |label|
+------------------------------------+-----+
|(8,[1,5,6,7],[6.0,378.0,21.33,69.0])|1    |
|(8,[1,5,6,7],[6.0,386.0,13.17,68.0])|1    |
|(8,[1,5,6,7],[6.0,538.0,20.0,84.0]) |1    |
+------------------------------------+-----+
only showing top 3 rows



### 2.5.6. Xây dựng model

In [26]:
lor_model.buildModel()

### 2.5.7. Đánh giá kết quả

In [27]:
lor_model.evaluateTestData()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1637|
|    0|       0.0| 2597|
|    1|       1.0| 3174|
|    0|       1.0| 2076|
+-----+----------+-----+



* Tính toán accuracy bằng tay

In [28]:
test_model = lor_model.model.transform(lor_model.test_data)

In [30]:
TN = test_model.filter('prediction = 0 AND label = prediction').count()
TP = test_model.filter('prediction = 1 AND label = prediction').count()
FN = test_model.filter('prediction = 0 AND label != prediction').count()
FP = test_model.filter('prediction = 1 AND label != prediction').count()

In [31]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)

In [32]:
precision, recall

(0.6045714285714285, 0.6597381001870712)

In [33]:
accuracy = (TN + TP) / (TN + TP + FN + FP)

In [34]:
accuracy

0.6084985238296078

* Đánh giá bằng thư viện

In [42]:
lor_model.evaluate()

,metric,value
0,weighted precision,0.608906
1,area under ROC,0.649156


### 2.5.8. Lưu và load model

#### 2.5.8.1. Lưu model

In [43]:
save_path = "./models/flights_LogisticRegression_0"

In [45]:
lor_model.saveModel(save_path)

#### 2.5.8.2. Load model

In [46]:
unlabeled_data = lor_model.test_data.select('features')

In [48]:
lor_model_1 = mySparkLogisticRegressionLoadModel(save_path)

In [49]:
predictions = lor_model_1.transform(unlabeled_data)

In [51]:
predictions.show(2, False)

+------------------------------------+----------------------------------------+----------------------------------------+----------+
|features                            |rawPrediction                           |probability                             |prediction|
+------------------------------------+----------------------------------------+----------------------------------------+----------+
|(8,[1,5,6,7],[6.0,1157.0,6.5,118.0])|[0.060426214509542,-0.060426214509542]  |[0.5151019587240296,0.48489804127597036]|0.0       |
|(8,[1,5,6,7],[13.0,425.0,18.5,75.0])|[-0.8638590197782436,0.8638590197782436]|[0.2965337168863144,0.7034662831136855] |1.0       |
+------------------------------------+----------------------------------------+----------------------------------------+----------+
only showing top 2 rows

